In [1]:
PUZZLE_FILE = 'puzzles.csv'
PUZZLE_INFO_FILE = 'puzzle_info.csv'
SAMPLE_FILE = '7ocakbest.csv'
INCLUDES = [257,264, 265, 267, 270, 276, 277]

In [2]:
#  285, 286, 287, 291, 298,
#        309, 310, 319, 321, 328, 336, 337, 342, 351, 352, 353, 357, 358, 359,
#        360, 370, 372, 373, 381, 383, 385, 386, 387, 388

In [3]:
class ExceedMaxSizeError(RuntimeError):
    pass

def get_shortest_path(
    moves: dict[str, tuple[int, ...]], K: int, max_size: int | None
) -> dict[tuple[int, ...], list[str]]:
    n = len(next(iter(moves.values())))

    state = tuple(range(n))
    cur_states = [state]

    shortest_path: dict[tuple[int, ...], list[str]] = {}
    shortest_path[state] = []

    for _ in range(100 if K is None else K):
        next_states = []
        for state in cur_states:
            for move_name, perm in moves.items():
                next_state = tuple(state[i] for i in perm)
                if next_state in shortest_path:
                    continue
                shortest_path[next_state] = shortest_path[state] + [move_name]

                if max_size is not None and len(shortest_path) > max_size:
                    raise ExceedMaxSizeError

                next_states.append(next_state)
        cur_states = next_states

    return shortest_path

def scrape_shortest_path(moves):
    K = 2
    while True:
        try:
            #shortest_path = get_shortest_path(moves, K, None if K == 2 else 100000)
            #shortest_path = get_shortest_path(moves, K, None if K == 2 else 500000)
            shortest_path = get_shortest_path(moves, K, None if K == 2 else 500000)
        except ExceedMaxSizeError:
            break
        K += 1
    print(f"K: {K}")
    print(f"Number of shortest_path: {len(shortest_path)}")   
    return K, shortest_path

In [4]:
#考虑要不要把submission的每个状态遍历,看看中途有没有已经到了允许的范围却没有停下来的.
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
puzzle_info=pd.read_csv(PUZZLE_INFO_FILE)
puzzles=pd.read_csv(PUZZLE_FILE)
submission=pd.read_csv(SAMPLE_FILE)
print(f"previous Score: {submission['moves'].str.split('.').apply(lambda x: len(x)).sum()}")
improve_moves=[]
good_rows = []
for id in range(len(submission)):
    init_state=puzzles.iloc[id]['initial_state'].split(";")
    end_state=puzzles.iloc[id]['solution_state'].split(";")
    moves=submission.iloc[id]['moves'].split(".")
    
    if id not in INCLUDES:
        improve_moves.append(".".join(moves))
        continue
    
    allowed_moves=eval(puzzle_info[puzzle_info['puzzle_type']==puzzles.iloc[id]['puzzle_type']]['allowed_moves'].values[0])
    for key in list(allowed_moves.keys()):#['f1',……] 
        #np.argsort:数组排序后的索引,比如s=[2,0,1]->s'=[1,2,0] 
        #原来是数组的2位置的数赋值到0的位置,现在是数组0位置的值赋值到2的位置.
        allowed_moves["-" + key] = list(np.argsort(allowed_moves[key]))
        
    K, shortest_path = scrape_shortest_path(allowed_moves)
    
    num_wildcards=puzzles.iloc[id]['num_wildcards']
    current_state=init_state.copy()
    mn_score = 1000000000
    mn_moves = None
    mn_state = None
    bsz = len(moves)
    wrows = []
    for i in range(len(moves)):
        move=moves[i]
        current_state= [current_state[i] for i in allowed_moves[move]]
        num_current_wildcards=sum(c!=e for c,e in zip(current_state,end_state))
        rw = {'i': i, 'w': num_current_wildcards}
        cur_state = current_state.copy()
        mn_wc = 10000000000
        mn_j = -1
        rw['i_mn'] = -1
        rw['w_mn'] = -1
        rwil = []
        rwmnl = []
        for ji in range(30):
            j = i + 1 + ji
            if j >= len(moves):
                break
            m = moves[j]
            cur_state= [cur_state[i] for i in allowed_moves[m]]
            num_current_wildcards_2=sum(c!=e for c,e in zip(cur_state, current_state))
            #rw['w' + str(ji + 1)] = num_current_wildcards_2
            if ji > K:
                if num_current_wildcards_2 <= num_wildcards:
                    rwil.append(j)
                    rwmnl.append(num_current_wildcards_2)
                    if num_current_wildcards_2 < mn_wc:
                        mn_wc = num_current_wildcards_2
                        mn_j = j
        if mn_j >= 0:
            rw['i_mn'] = mn_j
            rw['w_mn'] = mn_wc
        for wi in range(30):
            if wi < len(rwil):
                rw['i_' + str(wi)] = rwil[wi]
                rw['w_' + str(wi)] = rwmnl[wi]
            else:
                rw['i_' + str(wi)] = -1
                rw['w_' + str(wi)] = -1
        wrows.append(rw)
    wdf = pd.DataFrame(wrows)
    wdf = wdf.sort_values(by=['w'], ascending=[True])
    wdf.to_csv('wildcard-' + str(id) + '-1.csv', index=False)
    wdf2 = wdf[wdf['w_mn'] >= 0]
    if len(wdf2) > 0:
        wdf2 = wdf2.sort_values(by=['i_mn'], ascending=[True])
        wdf2.to_csv('wildcard-' + str(id) + '-2.csv', index=False)
    print('Smallest wildcards: ' + str(wdf['w'].iloc[0]) + ' / ' + str(num_wildcards) + ' --> ' + str(wdf['i'].iloc[0] + 1) + ' / ' + str(len(moves)))
    current_state=init_state.copy()   
    bs_i = -1
    for i in range(len(moves)):
        move=moves[i]
        current_state= [current_state[i] for i in allowed_moves[move]]
        #num_current_wildcards=sum(c!=e for c,e in zip(current_state,end_state))
            
        if i > bs_i and len(wdf2) > 0:
            df = wdf2[wdf2['i'] == i]
            if len(df) > 0:
                for wi in range(30):
                    if df['i_' + str(wi)].iloc[0] >= 0 and df['w_' + str(wi)].iloc[0] >= 0:
                        bs_moves_2 = moves[:i+1]
                        bs_score_2 = len(bs_moves_2)
                        bs_j_2 = df['i_' + str(wi)].iloc[0]
                        bs_w_2 = df['w_' + str(wi)].iloc[0]

                        left_state = current_state.copy()
                        right_state = left_state.copy()
                        z = i + 1
                        while z <= bs_j_2:
                            m = moves[z]
                            right_state= [right_state[i] for i in allowed_moves[m]]
                            z += 1
                            
                        if bs_w_2 == 0:
                            cur_moves = bs_moves_2.copy()                            
                            cur_moves_2 = cur_moves.copy() + moves[bs_j_2 + 1:]
                            asz = len(cur_moves_2)
                            if asz < bsz:
                                if asz < mn_score:
                                    print('=> [GF] ' + str(i) + ', ' + str(bs_j_2) + ' -> ' + str(bsz) + ' -> ' + str(asz))
                                    mn_score = asz
                                    mn_moves = cur_moves_2.copy()
                                    good = True
                                    if bs_j_2 > bs_i:
                                        bs_i = bs_j_2
                        else:
                            good = False
                            for perm, move_names in shortest_path.items():
                                cur_state = left_state.copy()
                                cur_moves = bs_moves_2.copy()
                                for m in move_names:
                                    cur_state= [cur_state[j] for j in allowed_moves[m]]
                                    num_current_wildcards=sum(c!=e for c,e in zip(cur_state,right_state))
                                    cur_moves.append(m)
                                    if num_current_wildcards == 0:
                                        cur_moves_2 = cur_moves.copy() + moves[bs_j_2 + 1:]
                                        asz = len(cur_moves_2)
                                        if asz < bsz:
                                            if asz < mn_score:
                                                print('=> [GF] ' + str(i) + ', ' + str(bs_j_2) + ' -> ' + str(bsz) + ' -> ' + str(asz))
                                                mn_score = asz
                                                mn_moves = cur_moves_2.copy()
                                                good = True
                                                if bs_j_2 > bs_i:
                                                    bs_i = bs_j_2
                        if not good:
                            print('=> [NG] ' + str(i) + ', ' + str(bs_j_2) + ' -> ' + str(bsz))

    if mn_moves is not None:
        good = False
        current_state=init_state.copy()   
        for i in range(len(mn_moves)):
            move=mn_moves[i]
            current_state= [current_state[j] for j in allowed_moves[move]]
            num_current_wildcards=sum(c!=e for c,e in zip(current_state,end_state))
            if num_current_wildcards <= num_wildcards:
                mn_moves = mn_moves[:i+1]
                good = True
                break
        if not good:
            mn_moves = None
    if mn_moves is not None:
        asz = len(mn_moves)
        print('=> [' + str(id) + '] Success! ' + str(mn_score - bsz) + ' : ' + str(bsz) + ' -> ' + str(asz))
        improve_moves.append(".".join(mn_moves))
        good_rows.append({'id': id, 'moves': ".".join(mn_moves)})
    else:
        improve_moves.append(".".join(moves))
        
submission['moves']=improve_moves
submission.to_csv("submission.csv",index=None)
print(f"current Score: {submission['moves'].str.split('.').apply(lambda x: len(x)).sum()}")
submission.head()

previous Score: 827944
K: 4
Number of shortest_path: 50800
Smallest wildcards: 0 / 42 --> 588 / 588
K: 4
Number of shortest_path: 75817
Smallest wildcards: 0 / 34 --> 969 / 969
K: 4
Number of shortest_path: 75817
Smallest wildcards: 0 / 18 --> 935 / 935
K: 4
Number of shortest_path: 107938
Smallest wildcards: 0 / 38 --> 1331 / 1331
=> [NG] 562, 568 -> 1331
=> [NG] 1027, 1033 -> 1331
=> [NG] 1045, 1051 -> 1331
K: 4
Number of shortest_path: 107938
Smallest wildcards: 0 / 16 --> 1305 / 1305
K: 4
Number of shortest_path: 148051
Smallest wildcards: 40 / 54 --> 2099 / 2099
=> [NG] 1723, 1729 -> 2099
=> [NG] 1731, 1737 -> 2099
K: 3
Number of shortest_path: 10888


In [5]:
gdf = None
if len(good_rows) > 0:
    gdf = pd.DataFrame(good_rows)
    gdf.to_csv('goodwithwildcards.csv', index=False)
gdf

,id,moves
0,257,f0.f1.f2.-r0.-d6.-r4.-r5.-r6.f4.f5.f6.r0.r1.r2...
1,264,-d0.-d2.-d1.-d3.r0.r2.r1.r3.d0.d2.d1.f1.f0.f2....
2,267,d0.-f6.-f5.-f8.-f7.-f6.-f5.-f8.-f7.-r6.-r5.-r8...
3,276,d0.d1.d2.d3.d4.-r6.-r7.-r8.-r9.-r6.-r7.-r8.-r9...
4,277,f0.-d10.-d11.-d12.-d13.-d14.-d15.-d16.-d17.-d1...


In [6]:
df = submission
df['size'] = df['moves'].apply(lambda x: len(x.split('.')))
score = df['size'].sum()
print('Score: ' + str(score))
df = pd.DataFrame([{'score': score}])
df.to_csv('score.csv', index=False)
df

Score: 834578


,score
0,834578


In [7]:
submission

,id,moves,size
0,0,r1.-f1,2
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0,9
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1,12
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0,10
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1,12
...,...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...,29562
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...,28657
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....,28758
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...,26340


In [19]:
submission=submission.drop(columns='size',axis=1)

In [8]:
submission.to_csv('7ocakv2.csv',index=False)

In [12]:
submission

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...
